# Phase 5: Legacy Validation 사용 예제

Phase 5에서 구현한 Legacy 비교 검증 시스템의 사용법을 설명합니다.

## 구현된 기능
- BaseLegacyComparator (Template Method Pattern)
- ToyoCycleComparator (Toyo 사이클 비교)
- Tolerance-based Validation (6가지 물리량별 허용 오차)
- Column-wise Deviation Tracking (8개 컬럼)
- Legacy Wrapper (안전한 BatteryDataTool.py 호출)

**요구사항**: BatteryDataTool.py와 pyodbc 모듈이 필요합니다.

## 1. 환경 확인

Legacy 비교를 위한 환경을 확인합니다.

In [ ]:
import sys
sys.path.insert(0, '..')

from src.utils.legacy_wrapper import check_battery_data_tool_available

# BatteryDataTool.py 가용성 확인
legacy_available = check_battery_data_tool_available()

print(f"Legacy BatteryDataTool.py Available: {legacy_available}")

if not legacy_available:
    print("\n⚠️ Warning: BatteryDataTool.py not found")
    print("  Place BatteryDataTool.py in parent directory to enable legacy comparison")
    print("  Also ensure 'pyodbc' module is installed: pip install pyodbc")
else:
    print("\n✓ Legacy comparison system ready")

## 2. 간단한 비교 (Quick Compare)

가장 간단한 방법으로 Legacy와 비교합니다.

In [ ]:
from src.validation.toyo_cycle_comparator import quick_compare

# Quick comparison (자동 용량 계산)
result = quick_compare(
    raw_file_path="Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc/30",
    mincapacity=0,  # 자동 계산
    firstCrate=0.2,
    chkir=False,
    print_report=True  # 상세 리포트 출력
)

print(f"\nComparison Result: {'✓ PASSED' if result.passed else '✗ FAILED'}")

## 3. 상세 비교 설정

ComparisonConfig로 상세 설정을 지정합니다.

In [ ]:
from src.validation.base_comparator import ComparisonConfig
from src.validation.toyo_cycle_comparator import ToyoCycleComparator

# 비교 설정
config = ComparisonConfig(
    raw_file_path="Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc/30",
    mincapacity=0,
    firstCrate=0.2,
    chkir=False,
    tolerances=None  # None이면 기본 tolerance 사용
)

# Comparator 생성 및 실행
comparator = ToyoCycleComparator(config)
result = comparator.compare()

# 상세 리포트 출력
comparator.print_report(result)

## 4. Comparison Result 분석

ComparisonResult 객체의 다양한 정보를 확인합니다.

In [ ]:
# 기본 정보
print("Comparison Summary:")
print(f"  Total Comparisons: {result.total_comparisons}")
print(f"  Exact Matches: {result.exact_matches}")
print(f"  Within Tolerance: {result.within_tolerance}")
print(f"  Match Rate: {(result.within_tolerance/result.total_comparisons)*100:.1f}%")

# 용량 비교
print(f"\nCapacity Comparison:")
print(f"  Legacy: {result.capacity_legacy:.1f} mAh")
print(f"  New: {result.capacity_new:.1f} mAh")
print(f"  Match: {result.capacity_match}")

# 오차 메트릭
print(f"\nError Metrics:")
print(f"  Mean Absolute Error: {result.mean_absolute_error:.6f}")
print(f"  Max Deviation: {result.max_deviation:.6f}")

# 불일치 행
print(f"\nMismatched Rows: {len(result.mismatched_rows)}")
if result.mismatched_rows:
    print(f"  Indices: {result.mismatched_rows[:10]}...")

## 5. Column-wise Deviation 분석

각 컬럼별 편차를 상세히 분석합니다.

In [ ]:
import pandas as pd

# Column Deviation DataFrame 생성
deviation_data = []
for col, stats in result.column_deviations.items():
    deviation_data.append({
        "Column": col,
        "Match %": stats["match_percentage"],
        "MAE": stats["mean_abs_error"],
        "Max Dev": stats["max_deviation"],
        "Tolerance": stats["tolerance"]
    })

deviation_df = pd.DataFrame(deviation_data)
print("Column-wise Deviation Analysis:")
print(deviation_df.to_string(index=False))

# 가장 정확한 컬럼
best_col = deviation_df.loc[deviation_df["MAE"].idxmin()]
print(f"\nBest Match Column: {best_col['Column']}")
print(f"  MAE: {best_col['MAE']:.6f}")
print(f"  Match Rate: {best_col['Match %']:.1f}%")

## 6. Tolerance 커스터마이징

물리량별 허용 오차를 커스터마이징할 수 있습니다.

In [ ]:
# 기본 tolerance 확인
default_tolerances = comparator._default_tolerances()

print("Default Tolerances:")
for key, value in default_tolerances.items():
    print(f"  {key}: ±{value}")

# Custom tolerance 적용
custom_tolerances = {
    "capacity": 0.05,      # 더 엄격하게 (기본: 0.1)
    "efficiency": 0.002,   # 더 관대하게 (기본: 0.001)
    "voltage": 0.001,
    "energy": 0.1,
    "temperature": 0.1,
    "dcir": 0.01
}

config_custom = ComparisonConfig(
    raw_file_path="Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc/30",
    mincapacity=0,
    firstCrate=0.2,
    chkir=False,
    tolerances=custom_tolerances
)

comparator_custom = ToyoCycleComparator(config_custom)
result_custom = comparator_custom.compare()

print(f"\nCustom Tolerance Result: {'✓ PASSED' if result_custom.passed else '✗ FAILED'}")
print(f"  Match Rate: {(result_custom.within_tolerance/result_custom.total_comparisons)*100:.1f}%")

## 7. 수동 용량 지정 검증

수동으로 지정한 용량이 Legacy와 동일하게 사용되는지 검증합니다.

In [ ]:
# 수동 용량 지정
config_manual = ComparisonConfig(
    raw_file_path="Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc/30",
    mincapacity=2000.0,  # 수동 지정
    firstCrate=0.2,
    chkir=False
)

comparator_manual = ToyoCycleComparator(config_manual)
result_manual = comparator_manual.compare()

print("Manual Capacity Validation:")
print(f"  Specified: {config_manual.mincapacity} mAh")
print(f"  Legacy Used: {result_manual.capacity_legacy} mAh")
print(f"  New Used: {result_manual.capacity_new} mAh")
print(f"  All Match: {result_manual.capacity_match and result_manual.capacity_legacy == config_manual.mincapacity}")

## 8. Result Export

ComparisonResult를 Dict/JSON으로 내보낼 수 있습니다.

In [ ]:
import json

# Dictionary로 변환
result_dict = result.to_dict()

print("Result Dictionary Structure:")
print(f"  Keys: {list(result_dict.keys())}")

# JSON 저장
with open('comparison_result.json', 'w', encoding='utf-8') as f:
    json.dump(result_dict, f, indent=2, ensure_ascii=False)

print("\n✓ Result exported to comparison_result.json")

# Summary 정보 확인
print("\nSummary:")
print(json.dumps(result_dict["summary"], indent=2))

## 9. 다중 경로 검증

여러 경로에 대해 일괄 검증을 수행합니다.

In [ ]:
import os

# 검증할 경로 목록
test_paths = [
    ("Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc", "30"),
    ("Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc", "31"),
    ("Rawdata/250219_250319_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 101-200cyc", "30"),
]

validation_results = []

for path, channel in test_paths:
    full_path = f"{path}/{channel}"
    
    # 경로 존재 확인
    if not os.path.exists(full_path):
        print(f"⚠️ Skipping {full_path} (not found)")
        continue
    
    # 비교 실행
    result = quick_compare(
        raw_file_path=full_path,
        mincapacity=0,
        firstCrate=0.2,
        chkir=False,
        print_report=False
    )
    
    validation_results.append({
        "path": full_path,
        "passed": result.passed,
        "cycles": result.total_comparisons,
        "match_rate": (result.within_tolerance/result.total_comparisons)*100,
        "mae": result.mean_absolute_error
    })
    
    status = "✓" if result.passed else "✗"
    print(f"{status} {full_path.split('/')[-2]}/{channel}: {result.total_comparisons} cycles, MAE={result.mean_absolute_error:.6f}")

# 전체 결과
total_passed = sum(1 for r in validation_results if r["passed"])
print(f"\n✓ Validation Complete: {total_passed}/{len(validation_results)} passed")

## 10. Tolerance별 민감도 분석

다양한 tolerance 값에 대한 검증 결과를 비교합니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 다양한 capacity tolerance 테스트
tolerance_values = [0.01, 0.05, 0.1, 0.2, 0.5, 1.0]
match_rates = []

for tol in tolerance_values:
    custom_tol = {
        "capacity": tol,
        "efficiency": 0.001,
        "voltage": 0.001,
        "energy": 0.1,
        "temperature": 0.1,
        "dcir": 0.01
    }
    
    config = ComparisonConfig(
        raw_file_path="Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc/30",
        mincapacity=0,
        firstCrate=0.2,
        chkir=False,
        tolerances=custom_tol
    )
    
    comparator = ToyoCycleComparator(config)
    result = comparator.compare()
    
    match_rate = (result.within_tolerance / result.total_comparisons) * 100
    match_rates.append(match_rate)
    
    print(f"Tolerance ±{tol}: {match_rate:.1f}% match")

# 시각화
plt.figure(figsize=(10, 6))
plt.plot(tolerance_values, match_rates, 'o-', linewidth=2, markersize=8)
plt.xlabel('Capacity Tolerance', fontsize=12)
plt.ylabel('Match Rate (%)', fontsize=12)
plt.title('Tolerance Sensitivity Analysis', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axhline(y=100, color='g', linestyle='--', alpha=0.5, label='100% match')
plt.legend()
plt.tight_layout()
plt.show()

# 권장 tolerance
recommended_idx = np.where(np.array(match_rates) >= 99.9)[0]
if len(recommended_idx) > 0:
    recommended_tol = tolerance_values[recommended_idx[0]]
    print(f"\n💡 Recommended capacity tolerance: ±{recommended_tol} (≥99.9% match)")

## 요약

Phase 5 Legacy Validation은:

1. **Template Method Pattern**: 체계적인 비교 프로세스
2. **Tolerance-based**: 물리량별 맞춤 허용 오차 (6종류)
3. **Column-wise Tracking**: 8개 컬럼별 상세 편차 분석
4. **Safe Wrapper**: 안전한 Legacy 함수 호출
5. **결과 Export**: Dict/JSON으로 결과 저장
6. **Quick Compare**: 간편한 일괄 검증
7. **100% 호환성**: Legacy와 동일한 출력 보장

**권장 Tolerance 값**:
- capacity: ±0.1 (normalized)
- efficiency: ±0.001 (±0.1%)
- voltage: ±0.001 V (±1 mV)
- energy: ±0.1 mWh
- temperature: ±0.1 °C
- dcir: ±0.01 mΩ

다음 Phase에서는 통합 테스트와 Production 배포 준비를 완료합니다.